In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from matplotlib.pyplot import figure
import seaborn as sns
import time, timeit
from cdlib import algorithms
import networkx as nx
from networkx.algorithms.community import louvain_communities, \
    label_propagation_communities, partition_quality, lukes_partitioning, \
        modularity, greedy_modularity_communities, naive_greedy_modularity_communities, girvan_newman
from networkx.generators.community import LFR_benchmark_graph
import community
import community.community_louvain
from sklearn.metrics.cluster import normalized_mutual_info_score
from scipy import stats

import random
from numpy import random as nprand
random.seed(42)
nprand.seed(42)

Note: to be able to use all crisp methods, you need to install some additional packages:  {'karateclub', 'infomap', 'wurlitzer', 'graph_tool'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'infomap', 'wurlitzer'}


In [31]:
# For plotting the community 
def plot_community(G, communities):
    def set_node_community(G, communities):
        '''Add community to node attributes'''
        for c, v_c in enumerate(communities):
            for v in v_c:
                # Add 1 to save 0 for external edges
                G.nodes[v]['community'] = c + 1
    def set_edge_community(G):
        '''Find internal edges and add their community to their attributes'''
        for v, w, in G.edges:
            if G.nodes[v]['community'] == G.nodes[w]['community']:
                # Internal edge, mark with community
                G.edges[v, w]['community'] = G.nodes[v]['community']
            else:
                # External edge, mark as 0
                G.edges[v, w]['community'] = 0
    def get_color(i, r_off=1, g_off=1, b_off=1):
        '''Assign a color to a vertex.'''
        r0, g0, b0 = 0, 0, 0
        n = 16
        low, high = 0.1, 0.9
        span = high - low
        r = low + span * (((i + r_off) * 3) % n) / (n - 1)
        g = low + span * (((i + g_off) * 5) % n) / (n - 1)
        b = low + span * (((i + b_off) * 7) % n) / (n - 1)
        return (r, g, b)
    
    spring_pos = nx.spring_layout(G) 
    # Set node and edge communities
    set_node_community = set_node_community(G, communities)
    set_edge_community = set_edge_community(G)
    node_color = [get_color(G.nodes[v]['community']) for v in G.nodes]
    # Set community color for edges between members of the same community (internal) and intra-community edges (external)
    external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] == 0]
    internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 0]
    internal_color = ['black' for e in internal]
    # external edges
    nx.draw_networkx(
        G,
        pos=spring_pos,
        node_size=0,
        edgelist=external,
        edge_color="silver",
        node_color=node_color,
        alpha=0.5,
        with_labels=True)
    nx.draw_networkx(
        G, pos=spring_pos,
        edgelist=internal,
        edge_color=internal_color,
        node_color=node_color,
        alpha=1,
        with_labels=True)
    return plt.draw()

def plt_com(G, communities, algo, dataset):
    plt.figure()
    plt.title(f'{algo} Algorithm - {dataset}')
    plot_community(G, communities)
    plt.show()

In [32]:
def find_community(G, algo):
    if algo == 'Louvain':
        start_time = time.time()
        # communities = louvain_communities(G, weight='weight', resolution=1, threshold=1e-07, seed=42)
        comm = algorithms.louvain(G.to_undirected(), weight='weight', resolution=1.0, randomize=True)
        communities = comm.communities
        end_time = time.time()
    elif algo == 'Greedy_Modularity':
        start_time = time.time()
        communities = sorted(greedy_modularity_communities(G), key=len, reverse=True)
        end_time = time.time()
    elif algo == 'Leiden':        
        start_time = time.time()
        communities = algorithms.leiden(G)
        comm = algorithms.leiden(G)
        communities = comm.communities
        end_time = time.time()
    elif algo == 'Girvan_Newman':        
        start_time = time.time()
        comp = girvan_newman(G)
        communities = tuple(sorted(c) for c in next(comp))
        end_time = time.time()
    elif algo == 'Naive_Greedy':        
        start_time = time.time()
        communities = naive_greedy_modularity_communities(G.to_undirected())
        end_time = time.time()
    elif algo == 'WalkTrap':        
        start_time = time.time()
        comm = algorithms.walktrap(G)
        communities = comm.communities
        end_time = time.time()
    elif algo == 'Surprise':        
        start_time = time.time()
        comm = algorithms.surprise_communities(G)
        communities = comm.communities
        end_time = time.time()
    total_time = end_time-start_time
    return communities, total_time

In [33]:
def find_modularity(G, communities):
    return modularity(G, communities)

In [34]:
def find_quality(G, communities):
    partition_quality_score = partition_quality(G, communities)
    return partition_quality_score

In [35]:
def dataset_desc(G, dataset):

    print(f'Dataset Description of {dataset}:')
    print(f'Number of Nodes in the network: {len(G.nodes())}')
    print(f'Number of Edges in the network: {len(G.edges())}\n') 
    
    # WCC = nx.weakly_connected_components(G)
    # print(f"Weakly connected component in small graph : {len(list(WCC))}" )
    # SCC = nx.strongly_connected_components(G)
    # print(f"Strongly connected component in small graph : {len(list(SCC))}" )

    # # small Dataset - Largest Strongly Connected component
    # largest_SCC = max(nx.strongly_connected_components(G), key=len)
    # largest_SCC_subgraph= G.subgraph(largest_SCC)
    # print(f"Number of nodes the largest strongly connected component : {len(nx.nodes(largest_SCC_subgraph))}")
    # print(f"Number of links the largest strongly connected component : {len(nx.edges(largest_SCC_subgraph))}")

    # # small Dataset - Largest Weakly Connected component
    # largest_WCC = max(nx.weakly_connected_components(G), key=len)
    # largest_WCC_subgraph= G.subgraph(largest_WCC)
    # print(f"Number of nodes the largest weakly connected component : {len(nx.nodes(largest_WCC_subgraph))}")
    # print(f"Number of links the largest weakly connected component : {len(nx.edges(largest_WCC_subgraph))}")

In [ ]:
def performance_testing(G, algo, dataset):
    communities, run_time = find_community(G, algo)
    print(f'{algo} Algorithm performance on {dataset} Dataset:')
    print(f'Communities found : {len(communities)} ')
    print(f'Modularity : {modularity(G, communities):.5f}')
    # coverage, performance = find_quality(G, communities)
    # print(f'Coverage : {coverage:.5f}')
    # print(f'Performance : {performance:.5f}')
    print(f'Computation Time Taken to find communities: {run_time:.2f}s\n')
    # print(f'Community Detection Visulaization of {algo} algorithm :')
    # plt_com(G, communities,algo, dataset)

In [37]:
# Dataset
karate_club = nx.karate_club_graph()

df_amazon = pd.read_csv('./data/Amazon0302.txt', sep='\t')
amazon_dataset = nx.from_pandas_edgelist(df_amazon, create_using=nx.DiGraph())

df_webgraph = pd.read_csv('./data/web-Google.txt', sep='\t')
web_graph = nx.from_pandas_edgelist(df_webgraph, create_using=nx.DiGraph())

df_biketheft = pd.read_csv('./data/bike_theft/bike_theft_data.csv', sep=',') # over last  10 years
bike_theft = nx.from_pandas_edgelist(df_biketheft, create_using=nx.DiGraph())

df_biketravel = pd.read_csv('./data/bike_travel/bike_travel_edge_list.csv')
bike_travel = nx.from_pandas_edgelist(df_biketravel, create_using=nx.DiGraph())

In [41]:
df_amazon_meta = pd.read_csv('./data/amazon_meta/amazon_edge_list.csv')
amazon_meta = nx.from_pandas_edgelist(df_amazon_meta[:881736], create_using=nx.DiGraph())

In [ ]:
dataset = ['Karate Club', 'Amazon CoPurchase', 'Google WebGraph', 'Bike Theft Dataset', 'Bike Travel', 'Amazon Meta Dataset']
algorithm = ['Louvain', 'Greedy_Modularity', 'Label_Propagation', 'Girvan_Newman', 'Naive_Greedy', 'WalkTrap', 'Surprise']

In [121]:
# Karate Club
dataset_desc(karate_club, dataset[0])
performance_testing(karate_club, 'Louvain', dataset[0])
performance_testing(karate_club, 'Greedy_Modularity', dataset[0])
performance_testing(karate_club, 'Leiden', dataset[0])
performance_testing(karate_club, 'Girvan_Newman', dataset[0])
performance_testing(karate_club, 'Naive_Greedy', dataset[0])
performance_testing(karate_club, 'WalkTrap', dataset[0])
performance_testing(karate_club, 'Surprise', dataset[0]) 

Dataset Description of Karate Club:
Number of Nodes in the network: 34
Number of Edges in the network: 78

Louvain Algorithm performance on Karate Club Dataset:
Communities found : 4 
Modularity : 0.44385
Coverage : 0.75641
Performance : 0.78253
Computation Time Taken to find communities: 0.00s

Greedy_Modularity Algorithm performance on Karate Club Dataset:
Communities found : 3 
Modularity : 0.41096
Coverage : 0.75641
Performance : 0.71480
Computation Time Taken to find communities: 0.00s

Leiden Algorithm performance on Karate Club Dataset:
Communities found : 4 
Modularity : 0.44490
Coverage : 0.73077
Performance : 0.80392
Computation Time Taken to find communities: 0.00s

Girvan_Newman Algorithm performance on Karate Club Dataset:
Communities found : 2 
Modularity : 0.34766
Coverage : 0.87179
Performance : 0.61141
Computation Time Taken to find communities: 0.02s

Naive_Greedy Algorithm performance on Karate Club Dataset:
Communities found : 3 
Modularity : 0.41096
Coverage : 0.75

In [142]:
# Amazon Dataset
dataset_desc(amazon_dataset, dataset[1])
performance_testing(amazon_dataset, 'Louvain', dataset[1])
# performance_testing(amazon_dataset, 'Greedy_Modularity', dataset[1]) # greedy won't perform on Amazon dataset, running for more than 24hrs...
performance_testing(amazon_dataset, 'Leiden', dataset[1])
# performance_testing(amazon_dataset, 'Girvan_Newman', dataset[1])
# performance_testing(amazon_dataset, 'Naive_Greedy', dataset[1])
# performance_testing(amazon_dataset, 'WalkTrap', dataset[1])
# performance_testing(amazon_dataset, 'Surprise', dataset[1])

Dataset Description of Amazon CoPurchase:
Number of Nodes in the network: 262111
Number of Edges in the network: 1234877

Louvain Algorithm performance on Amazon CoPurchase Dataset:
Communities found : 167 
Modularity : 0.91357
Coverage : 0.93446
Performance : 0.97927
Computation Time Taken to find communities: 74.41s

Leiden Algorithm performance on Amazon CoPurchase Dataset:
Communities found : 446 
Modularity : 0.05625
Coverage : 0.07104
Performance : 0.98570
Computation Time Taken to find communities: 21.15s



In [ ]:
# Google Web Graph Dataset
dataset_desc(web_graph, dataset[2])
# performance_testing(web_graph, 'Louvain', dataset[2])
# performance_testing(web_graph, 'Greedy_Modularity', dataset[2]) # greedy won't perform on Amazon dataset, running for more than 24hrs...
performance_testing(web_graph, 'Leiden', dataset[2])
performance_testing(web_graph, 'Girvan_Newman', dataset[2])
performance_testing(web_graph, 'Naive_Greedy', dataset[2])
performance_testing(web_graph, 'WalkTrap', dataset[2])
performance_testing(web_graph, 'Surprise', dataset[2])

In [140]:
# Bike Theft dataset
dataset_desc(bike_theft, dataset[3])
performance_testing(bike_theft, 'Louvain', dataset[3])
# performance_testing(bike_theft, 'Greedy_Modularity', dataset[3])
# performance_testing(bike_theft, 'Leiden', dataset[3])
# performance_testing(bike_theft, 'Girvan_Newman', dataset[3])
# performance_testing(bike_theft.to_undirected(), 'Naive_Greedy', dataset[3])
# performance_testing(bike_theft, 'WalkTrap', dataset[3])
# performance_testing(bike_theft, 'Surprise', dataset[3])

Dataset Description of Bike Theft Dataset:
Number of Nodes in the network: 18130
Number of Edges in the network: 17640

Louvain Algorithm performance on Bike Theft Dataset Dataset:
Communities found : 490 
Modularity : 0.99796
Coverage : 1.00000
Performance : 0.99807
Computation Time Taken to find communities: 0.43s



In [135]:
# Bike Travel Dataset
dataset_desc(bike_travel, dataset[4])
performance_testing(bike_travel, 'Louvain', dataset[4])
performance_testing(bike_travel, 'Greedy_Modularity', dataset[4])
performance_testing(bike_travel, 'Leiden', dataset[4])
performance_testing(bike_travel, 'Girvan_Newman', dataset[4])
# performance_testing(bike_travel.to_undirected(), 'Naive_Greedy', dataset[4])
performance_testing(bike_travel, 'WalkTrap', dataset[4])
performance_testing(bike_travel, 'Surprise', dataset[4])

Dataset Description of Bike Travel:
Number of Nodes in the network: 794
Number of Edges in the network: 277852

Louvain Algorithm performance on Bike Travel Dataset:
Communities found : 3 
Modularity : 0.20801
Coverage : 0.69975
Performance : 0.68496
Computation Time Taken to find communities: 2.38s

Greedy_Modularity Algorithm performance on Bike Travel Dataset:
Communities found : 3 
Modularity : 0.20824
Coverage : 0.70172
Performance : 0.68552
Computation Time Taken to find communities: 6.64s

Leiden Algorithm performance on Bike Travel Dataset:
Communities found : 3 
Modularity : 0.20912
Coverage : 0.64228
Performance : 0.68800
Computation Time Taken to find communities: 2.93s

Girvan_Newman Algorithm performance on Bike Travel Dataset:
Communities found : 2 
Modularity : 0.00000
Coverage : 1.00000
Performance : 0.44380
Computation Time Taken to find communities: 51.75s

WalkTrap Algorithm performance on Bike Travel Dataset:
Communities found : 5 
Modularity : 0.16990
Coverage : 0.

In [152]:
# Amazon Meta-Datset Club
dataset_desc(amazon_meta, dataset[5])
performance_testing(amazon_meta, 'Louvain', dataset[5])
performance_testing(amazon_meta, 'Greedy_Modularity', dataset[5])
performance_testing(amazon_meta, 'Leiden', dataset[5])
performance_testing(amazon_meta, 'Girvan_Newman', dataset[5])
performance_testing(amazon_meta, 'Naive_Greedy', dataset[5])
performance_testing(amazon_meta, 'WalkTrap', dataset[5])
performance_testing(amazon_meta, 'Surprise', dataset[5])

Dataset Description of Amazon Meta Dataset:
Number of Nodes in the network: 479749
Number of Edges in the network: 881736

Louvain Algorithm performance on Amazon Meta Dataset Dataset:
Communities found : 6226 
Modularity : 0.89734
Coverage : 0.93282
Performance : 0.97185
Computation Time Taken to find communities: 300.74s



KeyboardInterrupt: 